In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from matplotlib import pyplot
from stop_words import get_stop_words
import joblib
from datetime import date
import sys

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from stop_words import get_stop_words

In [16]:
class DataHandler:
    
    """
        Récupération des data depuis le GCS Bucket
    """

    def __init__(self):
        self.df_vaccin = None 
        
    def get_data(self):
        """
            Récupération du 
        """
        self.df_vaccin= pd.read_csv("utils/vaccination_tweets.csv",sep=",",index_col=0)
        print("data chargés")
    
    def clean_text(self):
        self.df['text'] = df['text'].apply(lambda tweet: re.sub('[^A-Za-z]+', ' ', tweet.lower()))

    def get_process_data(self):
        """
            Lancement des différente méthode get_data()+goup_data()
        """
        print('Data processed')
        return self.get_data()

In [17]:
data = DataHandler()
data.get_process_data()
%time

Data processed
data chargés
Wall time: 0 ns


In [18]:
class FeatureRecipe:
    
    def __init__(self, data: pd.DataFrame()):
        self.df_data = data
        self.cate = []
        self.floa = []
        self.intt = []
        self.drop = []
        

    def separate_variable_types(self) -> None:
        """ TODO : Diviser les types de variables dans un tableau """
        #variables = self.df_data.dtypes
        #var_tab = Arrays('int64',['125946'])
        #print(var_tab[int64])
        
        """correction"""
        print("separating columns")
        for col in self.df_data.columns:
            if self.df_data[col].dtypes == int:
                self.intt.append(self.df_data[col])
            elif self.df_data[col].dtypes == float:
                self.floa.append(self.df_data[col])
            else:
                self.cate.append(self.df_data[col])
        print ("dataset column size : {} \nnumber of discreet values : {} \nnumber of continuous values : {} \nnumber of others : {} \ntaille total : {}".format(len(self.df_data.columns),len(self.intt),len(self.floa),len(self.cate),len(self.intt)+len(self.floa)+len(self.cate) ))
    
    
    def drop_uselessf(self):
        """ TODO : Supprimer les colonnes inutiles du dataset """
        colonnes_drop=['user_created',
                       'user_followers',
                       'user_friends',
                       'user_favourites',
                      ]
        self.df_data.drop(columns=colonnes_drop)
        print('colonnes supprimer')
    
    def drop_duplicate(self):
        """ TODO : Supprimer les lignes dupliquées du dataset """
        print('duplicate')
        a=0
        for i in self.df_data:
            a+=1
            b=0
            for j in self.df_data:
                b +=1
                if a != b and self.df_data[i].equals(self.df_data[j]) == True:
                    self.df_data.drop(columns=j)
                    print('{} supprimée'.format(j))
    
    def Verif_data(self):
        print('verif date')
        """ Vérif des data supérieur a 3%"""
        for colonne in  self.df_data:
            nbNaN =  self.df_data[colonne].isna().sum()
            if (nbNaN /  self.df_data.shape[0]) * 100 > 3:
                del  self.df_data[colonne]
                print('{} supprimée'.format(colonne))
        


    def prepare_data(self):
        self.separate_variable_types()
        self.drop_uselessf()
        self.drop_duplicate()
        self.Verif_data()
        return self.df_data

In [19]:
data = DataHandler()
data.get_process_data()
feature = FeatureRecipe(data.df_vaccin)
feature.prepare_data()

Data processed
data chargés
separating columns
dataset column size : 15 
number of discreet values : 0 
number of continuous values : 0 
number of others : 15 
taille total : 15
colonnes supprimer
duplicate
verif date
user_location supprimée
user_description supprimée
hashtags supprimée


,user_name,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,source,retweets,favorites,is_retweet
id,,,,,,,,,,,,
1340539111971516416,Rachel Roh,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,Twitter for Android,0,0,False
1338158543359250433,Albert Fong,2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,While the world has been on the wrong side of ...,Twitter Web App,1,1,False
1337858199140118533,eli🇱🇹🇪🇺👌,2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,Twitter for Android,0,0,False
1337855739918835717,Charles Adler,2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're...",Twitter Web App,446,2129,False
1337854064604966912,Citizen News Channel,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @Bor...,Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1342992094961541122,Nidhi Palacios,2012-12-01 06:30:57,63,358,719,False,2020-12-27 00:34:01,Got my #COVID19 vaccine 💉 early morning on Dec...,Twitter for iPhone,0,0,False
1342990731850481667,Ramona,2020-12-20 14:55:06,2,16,59,False,2020-12-27 00:28:36,Where is #tiffanydover ? Why no one has news f...,Twitter for Android,0,3,False
1342988932615647233,AlejandroBerlino,2020-05-02 11:29:19,16,66,116,False,2020-12-27 00:21:27,BILL GATES &amp; WALTER WHITE ON - BREAKING GA...,Twitter Web App,0,0,False


In [20]:
class FeatureExtractor:
    """
    Feature Extractor class
    """
    def __init__(self, data: pd.DataFrame, flist: list = None):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split

        """
        self.df_data = data
        self.X = self.df_data['text']
        self.y = self.df_data['retweets']
        self.clf = None
        
    def make_pipeline(self):
        self.clf = make_pipeline(
                            TfidfVectorizer(stop_words=get_stop_words('en')),
                            OneVsRestClassifier(SVC(kernel='linear', probability=True))
                            )
        
        self.clf = self.clf.fit(X=self.X, y=self.y)
        
        return self.clf

In [21]:
data = DataHandler()
data.get_process_data()
feature = FeatureRecipe(data.df_vaccin)
feature.prepare_data()
train = FeatureExtractor(feature.df_data)

Data processed
data chargés
separating columns
dataset column size : 15 
number of discreet values : 0 
number of continuous values : 0 
number of others : 15 
taille total : 15
colonnes supprimer
duplicate
verif date
user_location supprimée
user_description supprimée
hashtags supprimée


In [22]:
class ModelBuild: 
    def __init__(self, model_path, save, n_estimators):
        """
        constructeur 
        """
        self.model_filename = model_path
        self.saveModel = save
        self.date = date.today().isoformat()
        self.n_estimators=n_estimators
    
    def train(self,clf,X,Y):
        clf.fit(X, Y)

    def predict_test(self,clf,text):
        #test 
        clf.predict_proba([text])[0]

    def print_accuracy(self,clf,X,Y):
        """
            affichage de la precision des predictions
        """
        accuracy=clf.score(X,Y)
        print('precision : {}'.format(accuracy))
    
    def FeatureImportance(self,X,Y,clf):
        """
            attribut un score aux valeurs utilisé pour la prediction basé 
            sur leurs utilité
        """
        clf.fit(X,Y)
        importance =  clf.coef_()
        for i,v in enumerate(importance):
            print('Feature: %0d, Score: %.5f' % (i,v))
        pyplot.bar([x for x in range(len(importance))], importance)
        pyplot.show()
        
    def save_model(self,clf):
        #save weights 
        model_filename = "retweet.joblib.z"
        joblib.dump((clf), model_filename)
        
    def calculData(self,clf,X,Y,text):
        self.train(clf,X,Y)
        self.predict_test(clf,text)
        self.print_accuracy(clf,X,Y)
        #self.FeatureImportance(X,Y,clf)
        self.save_model(clf)

In [23]:
data = DataHandler()
data.get_process_data()
feature = FeatureRecipe(data.df_vaccin)
feature.prepare_data()
train = FeatureExtractor(feature.df_data)
clf = train.make_pipeline()
modelbuild = ModelBuild("",True,50)
modelbuild.calculData(train.clf,train.X,train.y,"COUCOU")

Data processed
data chargés
separating columns
dataset column size : 15 
number of discreet values : 0 
number of continuous values : 0 
number of others : 15 
taille total : 15
colonnes supprimer
duplicate
verif date
user_location supprimée
user_description supprimée
hashtags supprimée


c:\python38\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


precision : 0.847757136384232
